In [ ]:
!pip install pymorphy2
!pip install stanza
!pip install detokenize

In [2]:
import csv
import pymorphy2
import stanza
from detokenize.detokenizer import detokenize

In [3]:
POS_TAGS = [
    "adj",
    "adv",
    "intj",
    "noun",
    "propn",
    "verb",
    "adp",
    "aux",
    "cconj",
    "det",
    "num",
    "part",
    "pron",
    "sconj",
    "x"
]

In [8]:
morph = pymorphy2.MorphAnalyzer()


class SearchEngineCSV:
    def __init__(self, csv_file):
        self.sentences = []
        self.tokens = []
        self.lemmas = []
        self.pos_tags = []
        self.chapters = []

        with open(csv_file, newline='', encoding='utf-8') as file:
            reader = csv.reader(file)
            next(reader)
            for row in reader:
                self.sentences.append(row[0])
                self.tokens.append([token.lower() for token in row[1].split(';')])  # Токены в нижнем регистре
                self.lemmas.append([lemma.lower() for lemma in row[2].split(';')])  # Леммы в нижнем регистре
                self.pos_tags.append(row[3].split(';'))
                self.chapters.append(row[4])

    def match_token(self, token, word, lemma, pos):
        token = token.lower()  # Приведение токена к нижнему регистру
        if '"' in token:
            return token.strip('"') == word

        if '+' in token:
            query_word, query_pos = token.split('+')
            return lemma.lower() == query_word.lower() and pos.lower() == query_pos.lower()

        if token in POS_TAGS:
            return pos.lower() == token

        return lemma == morph.parse(token)[0].normal_form

    def match_sequence(self, query_tokens, words, lemmas, pos_tags):
        if len(query_tokens)>2 and '"' in query_tokens[2]:
            query_tokens[1] = query_tokens[1] + '"'
        for j, query_token in enumerate(query_tokens):
            query_token = query_token.lower()  # Приведение токена запроса к нижнему регистру
            if query_token in POS_TAGS:
                if pos_tags[j].lower() != query_token:
                    return False
            else:
                if not self.match_token(query_token, words[j], lemmas[j], pos_tags[j]):
                    return False
        return True

    def search(self, query):
        query_tokens = query.lower().split()  # Приведение запроса к нижнему регистру

        matches = []
        for idx, sentence in enumerate(self.sentences):
            words = self.tokens[idx]
            lemmas = self.lemmas[idx]
            pos_tags = self.pos_tags[idx]
            if len(words) != len(lemmas) or len(words) != len(pos_tags):  # проверка, что количество токенов, лемм и тегов частей речи одинаковое
                continue

            # Поиск последовательности токенов в предложении
            for i in range(len(words) - len(query_tokens) + 1):
                formatted_sentence = ""
                matches_in_sent = []
                # Проверка последовательности для n-граммы
                if self.match_sequence(query_tokens, words[i:i+len(query_tokens)], lemmas[i:i+len(query_tokens)], pos_tags[i:i+len(query_tokens)]):
                    # Форматируем предложение с источником
                    source_info = f"[{self.chapters[idx]}]"
                    if formatted_sentence == "":
                        formatted_sentence = f"{sentence} {source_info}"
                    matches_in_sent.append((i, i + len(query_tokens)))  # информация о том, какие слова - ответ на запрос (номер первого слова последовательности и первого слова после неё)
                if formatted_sentence != "":
                    matches.append([formatted_sentence, matches_in_sent, idx])  # сначала строка "текст предложения [номер главы]", потом список мест, в которых слова, отвечающие на запрос

        return matches


In [21]:
def frequency(search_results, csv_file):  # функция, определяющая частотность, принимает результаты поиска
    tokens = []  # список списков токенов в каждом предложении
    with open(csv_file, newline='', encoding='utf-8') as file:
        reader = csv.reader(file)
        next(reader)
        for row in reader:
            tokens.append([token.lower() for token in row[1].split(';')])  # токены в нижнем регистре
    freq_list = {}  # вариант ответа на запрос : его частотность
    for results_elem in search_results:
        sent = results_elem[0]
        entry_indexes = results_elem[1]  # номера, на каких номерах слова, соответствующие самому запросу (номер первого слова и следующего после последнего слова)
        sent_num = results_elem[2]
        for i in entry_indexes:
            entry_tokens = tokens[sent_num][i[0]:i[1]]
            entry = detokenize(entry_tokens).replace('- ', '-')
            freq_list[entry] = freq_list.get(entry, 0) + 1
    freq_list_array = list(freq_list.items())
    freq_list_sorted =  sorted(freq_list_array, key=lambda x: x[1], reverse=True)
    return freq_list_sorted

In [9]:
csv_file = '/content/corpus_data_with_labels.csv'
search_engine = SearchEngineCSV(csv_file)

In [22]:
frequency(search_engine.search('NOUN'), csv_file)[:10]

[('глаза', 49),
 ('раз', 45),
 ('руку', 33),
 ('княжна', 29),
 ('человек', 27),
 ('день', 26),
 ('разговор', 25),
 ('сердце', 24),
 ('слова', 22),
 ('друг', 21)]

In [23]:
print(frequency(search_engine.search(input('Запрос для подсчёта частотностей: ')), csv_file))

Запрос для подсчёта частотностей: конь
[('коня', 10), ('конь', 4), ('конях', 1), ('коне', 1), ('кони', 1)]
